### Adattisztítás

flyball2024p3.jpynb

Az adattisztitás során figyelembevett szempontok:
- azon sorok eldobása, ahol sem név információ, sem futási érték nincs
- váltás oszlopokban az 'ok' értékeket nullával helyettesítjük, a különféle hibákat -999 -el, a NaN értékek a medián értékre cseréltem és a felső kiugró értékeket 1,7 sec felett hibának minősítettem
- time oszlopban a negatív időt, a hibajelzéseket és a valószínűtlenül jó futási időket (3,3 sec alatt) hibának minősítettem, ugyanígy a 10 sec feletti részidőket is
- when oszlop átalakítása csak óra-perc-másodperccé és a date oszlop hozzáadásával egy long_date oszlop létrehozása
- W/L/T oszlop NaN értékeit még korábban a széles adatbázisban feltöltöttem kód segítségével, az elnevezéseket egységesítettem

- a kutyára jellemző adatokat a FlyballKutya.jpynb-ben tisztítottam

In [1]:
import pandas as pd
szinesdf = pd.read_csv('./../data/szines2.csv', index_col=[0])


In [2]:
szinesdf.head()

,Division,when,colour,competition,date,place,who,with who,total time,W/L/T,Hurdles,rank,name,relay,time
0,13.0,1900-01-01 12:26:30,Blue,FLYBALL_EFC,2023-07-29,Tápiószentmárton Ring1,Real Ballance Flyball Team,WildRunners Zephyr,21.18,L,15.0,1,Chase,0.33,4.21
1,13.0,1900-01-01 12:26:30,Red,FLYBALL_EFC,2023-07-29,Tápiószentmárton Ring1,WildRunners Zephyr,Real Ballance Flyball Team,20.49,W,25.0,1,Brix,-0.03,4.58
2,13.0,1900-01-01 12:27:58,Blue,FLYBALL_EFC,2023-07-29,Tápiószentmárton Ring1,Real Ballance Flyball Team,WildRunners Zephyr,20.81,W,15.0,1,Chase,0.008,4.24
3,13.0,1900-01-01 12:27:58,Red,FLYBALL_EFC,2023-07-29,Tápiószentmárton Ring1,WildRunners Zephyr,Real Ballance Flyball Team,20.85,L,25.0,1,Brix,0.15,4.42
4,13.0,1900-01-01 12:29:11,Blue,FLYBALL_EFC,2023-07-29,Tápiószentmárton Ring1,Real Ballance Flyball Team,WildRunners Zephyr,20.66,W,15.0,1,Chase,0.09,4.24


In [39]:
szinesdf.isna().sum()
#nincs érték :
# 0 W/L/T
# 164 Hurdles
# 538 name
# 1080 relay
# 477 time

Division          0
when              0
colour            0
competition       0
date              0
place             0
who               0
with who          0
total time        0
W/L/T             0
Hurdles         164
rank              0
name            538
relay          1080
time            477
dtype: int64

In [3]:
#Dobjuk el azokat a sorokat, ahol nincs kiírva a kutya neve, hiszen így nem tudjuk az időeredményeket kutyákhoz rendelni:
szinesdf.loc[szinesdf['name'].isnull()].shape

szinesdf.dropna(subset=["name"], inplace=True)

In [41]:
szinesdf.isna().sum()
#nincs érték :
# 0 W/L/T
# 32 Hurdles
# 697 relay
# 97 time

Division         0
when             0
colour           0
competition      0
date             0
place            0
who              0
with who         0
total time       0
W/L/T            0
Hurdles         32
rank             0
name             0
relay          697
time            97
dtype: int64

In [4]:
#Dobjuk el azokat a sorokat, ahol nincs a kutya futási ideje megadva, mert nem tudjuk futási szempontból értékelni őket:
szinesdf.dropna(subset=["time"], inplace=True)


In [43]:
szinesdf.isna().sum()
#nincs érték :
# 0 W/L/T
# 32 Hurdles
# 610 relay

Division         0
when             0
colour           0
competition      0
date             0
place            0
who              0
with who         0
total time       0
W/L/T            0
Hurdles         32
rank             0
name             0
relay          610
time             0
dtype: int64

In [5]:
#Dobjuk el azokat a sorokat, ahol a kutya neve helyett egy 0 van:
szinesdf[szinesdf['name'] == '0'] #14 ilyen sor van
szinesdf.drop(szinesdf[szinesdf['name'] == '0'].index, inplace=True)

In [45]:
szinesdf.isna().sum()
#nincs érték :
# 0 W/L/T
# 24 Hurdles
# 610 relay

Division         0
when             0
colour           0
competition      0
date             0
place            0
who              0
with who         0
total time       0
W/L/T            0
Hurdles         24
rank             0
name             0
relay          610
time             0
dtype: int64

In [6]:
#relay oszlop vizsgálata:
#relay oszlopban milyen értékek vannak:
#van 610 NaN , 5546 OK, 419 Ok, 2390 ok, chyba 1452, error 162, vběhl 16

szinesdf.groupby('relay').count().tail(10)

,Division,when,colour,competition,date,place,who,with who,total time,W/L/T,Hurdles,rank,name,time
relay,,,,,,,,,,,,,,
9.87,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9.92,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9.96,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9.99,1,1,1,1,1,1,1,1,1,1,1,1,1,1
OK,5546,5546,5546,5546,5546,5546,5546,5546,5546,5546,5543,5546,5546,5546
Ok,419,419,419,419,419,419,419,419,419,419,419,419,419,419
chyba,1452,1452,1452,1452,1452,1452,1452,1452,1452,1452,1452,1452,1452,1452
error,162,162,162,162,162,162,162,162,162,162,159,162,162,162
ok,2390,2390,2390,2390,2390,2390,2390,2390,2390,2390,2388,2390,2390,2390


In [7]:
#relay oszlopokban a 3 féle formájú ok cseréljük 0-ra, mert ezek jó váltások, értékük lehet 0, a hibákat -999-re:
szinesdf['relay'] = szinesdf['relay'].replace('OK', 0) 
szinesdf['relay'] = szinesdf['relay'].replace('Ok', 0)
szinesdf['relay'] = szinesdf['relay'].replace('ok', 0)
szinesdf['relay'].replace('chyba', -999, inplace=True)
szinesdf['relay'].replace('error', -999, inplace=True)
szinesdf['relay'].replace('vběhl', -999, inplace=True)

In [8]:
#relay oszlopban már csak számok maradtak, ezért az oszlop adattípusát át tudjuk alakítani numerikussá
szinesdf['relay'] = pd.to_numeric(szinesdf['relay'])

#relay oszlopban maradt 610 db NaN érték
#ezeket cseréljük ki a relay oszlop median értékére, mert nem dobom el a futási eredmények megtartása miatt
#mean = 0.23399568446406777
#median = 0.13

szinesdf['relay'] = szinesdf['relay'].fillna(0.13)

#relay oszlopban hibakként kezeljük a felső kiugró értékeket is, 1,7 s felett, ide a váltásidők 3.4%-a tartozik
#ezeket -1 előjellel látjuk el, hogy majd egyszerre tudjuk a negatív értékeket hibának minősíteni
szinesdf.loc[szinesdf['relay'] > 1.7, 'relay'] = szinesdf['relay'] * (-1)

In [9]:
#relay oszlopban az összes hiba negatív értékű, ezek lekérése és lecserélése: 
szinesdf.loc[szinesdf['relay'] < 0, 'relay'] = 'error' 


In [10]:
#relay oszlopban a median érték kiszámolásához a NaN és az Error értéket tartalmazó sorokat nem veszzük figyelembe
hibanelkuli = szinesdf[(szinesdf['relay'] != 'error')]
hibaesnannelkul = hibanelkuli[hibanelkuli['relay'].notnull()]
hibaesnannelkul['relay'].median()


0.13

In [83]:
szinesdf.isna().sum()
#nincs érték :
# 0 W/L/T
# 24 Hurdles

Division        0
when            0
colour          0
competition     0
date            0
place           0
who             0
with who        0
total time      0
W/L/T           0
Hurdles        24
rank            0
name            0
relay           0
time            0
dtype: int64

In [11]:
#time oszlop
#itt is vannak hibajelzések, van negatív idő, és van valószínűtlen idő: 3,3 s alatt(outliers)
#hibák: OK (1), ok(1), chyba(1), mimo(312), outside(15), run in(5), vběhl(184), itt is -999-el helyettesitsük


szinesdf['time'].replace('OK', -999, inplace=True)
szinesdf['time'].replace('ok', -999, inplace=True)
szinesdf['time'].replace('mimo', -999, inplace=True)
szinesdf['time'].replace('outside', -999, inplace=True)
szinesdf['time'].replace('run in', -999, inplace=True)
szinesdf['time'].replace('vběhl', -999, inplace=True)
szinesdf['time'].replace('chyba', -999, inplace=True)

szinesdf.groupby('time').count().tail(3)

#Adattípus csere
szinesdf['time'] = pd.to_numeric(szinesdf['time'])

#time oszlopban hibakként kezeljük a felső kiugró értékeket is, 9 s felett
#ezeket -1 előjellel látjuk el, hogy majd egyszerre tudjuk a negatív értékeket hibának minősíteni
szinesdf.loc[szinesdf['time'] > 9, 'time'] = szinesdf['time'] * (-1)

In [12]:
#time oszlopban a 3,3 s alattiakat, így a -999-et is cseréljük error-ra, így csak időeredmények és 'error' felirat marad
szinesdf.loc[szinesdf['time'] < 3.3, 'time'] = 'error'

In [13]:
#a when oszlop alakítása, hogy egységes dátum legyen, a when oszlop csak hour:minute:second
szinesdf['when'] = pd.to_datetime(szinesdf['when'])
szinesdf['when'] = szinesdf['when'].dt.strftime('%H:%M:%S')
#új oszlopban a when és date egyesítsük hosszú dátummá
szinesdf['long_date'] = pd.to_datetime(szinesdf['when'] + ' ' + szinesdf['date'])

In [53]:
szinesdf.head()

,Division,when,colour,competition,date,place,who,with who,total time,W/L/T,Hurdles,rank,name,relay,time,long_date
0,13.0,12:26:30,Blue,FLYBALL_EFC,2023-07-29,Tápiószentmárton Ring1,Real Ballance Flyball Team,WildRunners Zephyr,21.18,L,15.0,1,Chase,0.33,4.21,2023-07-29 12:26:30
1,13.0,12:26:30,Red,FLYBALL_EFC,2023-07-29,Tápiószentmárton Ring1,WildRunners Zephyr,Real Ballance Flyball Team,20.49,W,25.0,1,Brix,error,4.58,2023-07-29 12:26:30
2,13.0,12:27:58,Blue,FLYBALL_EFC,2023-07-29,Tápiószentmárton Ring1,Real Ballance Flyball Team,WildRunners Zephyr,20.81,W,15.0,1,Chase,0.008,4.24,2023-07-29 12:27:58
3,13.0,12:27:58,Red,FLYBALL_EFC,2023-07-29,Tápiószentmárton Ring1,WildRunners Zephyr,Real Ballance Flyball Team,20.85,L,25.0,1,Brix,0.15,4.42,2023-07-29 12:27:58
4,13.0,12:29:11,Blue,FLYBALL_EFC,2023-07-29,Tápiószentmárton Ring1,Real Ballance Flyball Team,WildRunners Zephyr,20.66,W,15.0,1,Chase,0.09,4.24,2023-07-29 12:29:11


In [105]:
szinesdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33627 entries, 0 to 34275
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Division     33627 non-null  float64       
 1   when         33627 non-null  object        
 2   colour       33627 non-null  object        
 3   competition  33627 non-null  object        
 4   date         33627 non-null  object        
 5   place        33627 non-null  object        
 6   who          33627 non-null  object        
 7   with who     33627 non-null  object        
 8   total time   33627 non-null  float64       
 9   W/L/T        33627 non-null  object        
 10  Hurdles      33603 non-null  float64       
 11  rank         33627 non-null  int64         
 12  name         33627 non-null  object        
 13  relay        33627 non-null  object        
 14  time         33627 non-null  object        
 15  long_date    33627 non-null  datetime64[ns]
dtypes: d

In [14]:
#W/L/T oszlop nan értékeinek vizsgálata az összes kutyaváltó esetén
#Van-e még a tisztitás után maradt 25x4, azaz 100 sorból, ahol NON van?
#igen, maradt 42.
#Ezeknek a soroknak a 'colour' értéke is 0, ami azt jelenti, hogy a táblázatban egyedüli sorként szerepelnek 
# és nincs időpárjuk.

len(szinesdf[szinesdf['W/L/T'] == 'NON'] ) #42

szinesdf[szinesdf['W/L/T'] == 'NON']

,Division,when,colour,competition,date,place,who,with who,total time,W/L/T,Hurdles,rank,name,relay,time,long_date
1411,8.0,09:34:19,0,FLYBALL_CZ,2023-05-06,EagersCup_Zamberk_Ring1,RUNami B,WildRunners XXL,5.04,NON,17.5,1,Butch,0.09,4.95,2023-05-06 09:34:19
1426,8.0,09:52:01,0,FLYBALL_CZ,2023-05-06,EagersCup_Zamberk_Ring1,WildRunners Zephyr,Hopíci Brno,4.59,NON,25.0,1,Mason,0.16,4.42,2023-05-06 09:52:01
1467,1.0,11:08:50,0,FLYBALL_CZ,2023-05-06,EagersCup_Zamberk_Ring1,Akela Runners,Fractal International Affair,3.71,NON,20.0,1,Cassidy,0.11,3.6,2023-05-06 11:08:50
1478,1.0,11:23:30,0,FLYBALL_CZ,2023-05-06,EagersCup_Zamberk_Ring1,WildRunners hopefuls,WOOF ONCE,4.05,NON,27.5,1,Enola,0.11,3.94,2023-05-06 11:23:30
1537,1.0,13:37:22,0,FLYBALL_CZ,2023-05-06,EagersCup_Zamberk_Ring1,Flyvaryors New Wave,Akela Runners,3.77,NON,22.5,1,Toko,0.04,3.73,2023-05-06 13:37:22
2051,5.0,09:24:34,0,FLYBALL_CZ,2023-05-06,EagersCup_Zamberk_Ring2,WildRunners Maximini,Wing BEAST,0.00,NON,25.0,1,Gerry,error,error,2023-05-06 09:24:34
2353,5.0,17:34:37,0,FLYBALL_CZ,2023-05-06,EagersCup_Zamberk_Ring2,FOFR DOG,Wing BEAST,14.15,NON,22.5,1,Hessie,error,5.87,2023-05-06 17:34:37
2535,3.0,12:25:54,0,FLYBALL_CZ,2023-05-07,EagersCup_Zamberk_Ring2,Pilsen F.A.S.T.,RUNami A,0.00,NON,25.0,1,Jolly,error,error,2023-05-07 12:25:54
2536,3.0,12:27:40,0,FLYBALL_CZ,2023-05-07,EagersCup_Zamberk_Ring2,Pilsen F.A.S.T.,RUNami A,10.88,NON,25.0,1,Jolly,error,error,2023-05-07 12:27:40
2555,3.0,12:48:34,0,FLYBALL_CZ,2023-05-07,EagersCup_Zamberk_Ring2,X-treme,New Ballance Flyball Team,0.00,NON,30.0,1,Kerby,0.73,error,2023-05-07 12:48:34


In [55]:
szinesdf.isna().sum()

Division        0
when            0
colour          0
competition     0
date            0
place           0
who             0
with who        0
total time      0
W/L/T           0
Hurdles        24
rank            0
name            0
relay           0
time            0
long_date       0
dtype: int64

In [15]:
#W/L/T oszlop NaN értékeinek vizsgálata a WildRunners kutyaváltók esetén:

#WR kutyák kiválogatása:
wr_szines = szinesdf[szinesdf['who'].str.startswith('WildRunners')]
wr_szines[wr_szines['W/L/T'] == 'NON']

#Három olyan sor van, ahol nem volt időpárja és így nincs számitott érték sem:

,Division,when,colour,competition,date,place,who,with who,total time,W/L/T,Hurdles,rank,name,relay,time,long_date
1426,8.0,09:52:01,0,FLYBALL_CZ,2023-05-06,EagersCup_Zamberk_Ring1,WildRunners Zephyr,Hopíci Brno,4.59,NON,25.0,1,Mason,0.16,4.42,2023-05-06 09:52:01
1478,1.0,11:23:30,0,FLYBALL_CZ,2023-05-06,EagersCup_Zamberk_Ring1,WildRunners hopefuls,WOOF ONCE,4.05,NON,27.5,1,Enola,0.11,3.94,2023-05-06 11:23:30
2051,5.0,09:24:34,0,FLYBALL_CZ,2023-05-06,EagersCup_Zamberk_Ring2,WildRunners Maximini,Wing BEAST,0.00,NON,25.0,1,Gerry,error,error,2023-05-06 09:24:34


## Q/Race oszlop hozzáadás: 
- Domain kérdés-válasz: általában az első nap időmérő, a második nap éles verseny
- kivétel a júl.29-30-i bajnokság
- Q: első nap (időmérő), Race: második nap (versenynap), kivéve a júl 29-30, ahol mindkettő Race

In [109]:
szinesdf['date'].unique()

array(['2023-07-29', '2023-07-30', '2023-04-22', '2023-04-23',
       '2023-05-06', '2023-05-07', '2023-05-20', '2023-05-21',
       '2023-06-10', '2023-06-11', '2023-06-24', '2023-06-25',
       '2023-07-01', '2023-07-02', '2023-07-15', '2023-07-16',
       '2023-09-02', '2023-09-03', '2023-09-16', '2023-09-17',
       '2023-10-14', '2023-10-15'], dtype=object)

In [111]:
#Dictionary
datumok = szinesdf['date'].unique()
Q_Race = ['Race','Race','Q','Race',
        'Q','Race','Q','Race',
        'Q','Race','Q','Race',
        'Q','Race','Q','Race',
        'Q','Race','Q','Race', 'Q', 'Race']
dictQ = {'date': datumok, 'Q/Race':Q_Race}
QRace_df = pd.DataFrame(dictQ)

In [112]:
#Winner táblához merge:
szinesdf = pd.merge(szinesdf, QRace_df, on='date' )
szinesdf.shape

(33627, 17)

In [113]:
# Tápiószentmárton Ring1 Tápiószentmárton Ring2
# EagersCup_Zamberk_Ring1 EagersCup_Zamberk_Ring2
szinesdf.loc[szinesdf['place'] == 'Tápiószentmárton Ring1', 'place'] = 'Tápiószentmárton'
szinesdf.loc[szinesdf['place'] == 'Tápiószentmárton Ring2', 'place'] = 'Tápiószentmárton'

szinesdf.loc[szinesdf['place'] == 'EagersCup_Zamberk_Ring1', 'place'] = 'EagersCup_Zamberk'
szinesdf.loc[szinesdf['place'] == 'EagersCup_Zamberk_Ring2', 'place'] = 'EagersCup_Zamberk'

szinesdf.loc[szinesdf['place'] == 'Budeckej_Vodnany_Ring1', 'place'] = 'Budeckej_Vodnany'
szinesdf.loc[szinesdf['place'] == 'Budeckej_Vodnany_Ring2', 'place'] = 'Budeckej_Vodnany'

In [114]:
szinesdf['place'].unique()

array(['Tápiószentmárton', 'Heroltice', 'EagersCup_Zamberk', 'Flyvarors',
       'Medtrophy', 'BCCCZ_Prostejov', 'CrazyCup_NoveMesto',
       'Machale_Luonovice', 'Mistrovsy_Blsany', 'Krypky_Litomsyl',
       'Budeckej_Vodnany'], dtype=object)

In [117]:
szinesdf.tail()

,Division,when,colour,competition,date,place,who,with who,total time,W/L/T,Hurdles,rank,name,relay,time,long_date,Q/Race
33622,8.0,17:55:34,Red,FLYBALL_CZ,2023-10-15,Budeckej_Vodnany,FUNatic Jumpers,Akela Sparks,21.30,W,15.0,4,And,0.13,5.38,2023-10-15 17:55:34,Race
33623,8.0,17:57:19,Blue,FLYBALL_CZ,2023-10-15,Budeckej_Vodnany,Akela Sparks,FUNatic Jumpers,19.74,W,15.0,4,Zoya,0.38,error,2023-10-15 17:57:19,Race
33624,8.0,17:57:19,Red,FLYBALL_CZ,2023-10-15,Budeckej_Vodnany,FUNatic Jumpers,Akela Sparks,19.89,L,15.0,4,And,0.2,5.27,2023-10-15 17:57:19,Race
33625,8.0,17:59:17,Blue,FLYBALL_CZ,2023-10-15,Budeckej_Vodnany,Akela Sparks,FUNatic Jumpers,28.74,L,15.0,4,Zoya,0.48,4.87,2023-10-15 17:59:17,Race
33626,8.0,17:59:17,Red,FLYBALL_CZ,2023-10-15,Budeckej_Vodnany,FUNatic Jumpers,Akela Sparks,25.49,W,15.0,4,And,0.24,4.99,2023-10-15 17:59:17,Race


In [115]:
szinesdf.to_csv('./../data/szines2betti.csv')